In [1]:
pd.options.mode.chained_assignment = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#load Raw_counts.csv
df = pd.read_csv('Raw_Counts.csv')

In [3]:
df

,Ensembl_ID,SYMBOL,ENTREZID,GENENAME,F_IgE_S3,F_unstim_S4,M_IgE_S1,M_unstim_S2,M_IgE_S6,M_unstim_S7,F_IgE_S8,F_unstim_S9
0,ENSMUSG00000000001,Gnai3,14679.0,guanine nucleotide binding protein (G protein)...,7469,6485,5562,5949,12604,15952,13806,14898
1,ENSMUSG00000000003,Pbsn,54192.0,probasin,0,0,0,0,0,0,0,0
2,ENSMUSG00000000028,Cdc45,12544.0,cell division cycle 45,92,303,141,323,134,108,94,140
3,ENSMUSG00000000031,H19,14955.0,"H19, imprinted maternally expressed transcript",0,0,0,0,2,0,0,2
4,ENSMUSG00000000037,Scml2,107815.0,sex comb on midleg-like 2 (Drosophila),0,0,0,0,4,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
54227,ENSMUSG00000116995,NaN,NaN,NaN,106,142,78,122,238,445,286,401
54228,ENSMUSG00000116996,NaN,NaN,NaN,0,0,0,0,0,0,0,0
54229,ENSMUSG00000116997,NaN,NaN,NaN,0,0,0,0,0,0,0,0
54230,ENSMUSG00000116998,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [4]:
count_cols = ['F_IgE_S3', 'F_unstim_S4', 'M_IgE_S1', 'M_unstim_S2', 'F_IgE_S8', 'F_unstim_S9', 'M_IgE_S6', 'M_unstim_S7']

In [5]:
def preprocess(df):
    print('df dimensions:', df.shape)
    df = df.dropna()
    print('dropped unlabeled:', df.shape)
    df = df.replace(0, np.nan)
    df = df.dropna(axis=0, thresh = 2, subset=count_cols).fillna(0)
    print('dropped no reads:', df.shape)
    return df

In [6]:
df = preprocess(df)

df dimensions: (54232, 12)
dropped unlabeled: (24187, 12)
dropped no reads: (17401, 12)


In [7]:
def filtering(df, count_cols = count_cols, cpm = 1):
    min_cpm = df[count_cols].sum() * cpm / 1000000
    
    df_list = []
    for i in range(8):
        col_min = min_cpm[i]
        col = count_cols[i]
        df_list.append(df[col] > col_min)
        
    filtered_rows = pd.concat(df_list, join = 'outer', axis = 1).replace(False, np.nan).dropna().index
    df_filtered = df.loc[filtered_rows]
    print('filtered:', df_filtered.shape)
    return df_filtered

In [8]:
df = filtering(df)

filtered: (9784, 12)


In [9]:
df.describe()

,ENTREZID,F_IgE_S3,F_unstim_S4,M_IgE_S1,M_unstim_S2,M_IgE_S6,M_unstim_S7,F_IgE_S8,F_unstim_S9
count,9.784000e+03,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9.784000e+03,9784.000000,9.784000e+03
mean,1.187479e+06,1566.798242,1654.248160,1386.311325,1469.402085,2992.393806,3.645156e+03,2944.615699,3.481498e+03
std,1.038668e+07,7650.530926,7948.841014,7287.081181,9976.801571,11755.849442,1.712733e+04,10194.439813,1.611024e+04
min,1.130300e+04,16.000000,17.000000,14.000000,15.000000,30.000000,3.700000e+01,30.000000,3.500000e+01
25%,2.398875e+04,224.000000,278.000000,185.000000,175.000000,465.000000,5.927500e+02,460.750000,5.617500e+02
50%,6.906100e+04,551.000000,630.500000,485.000000,443.000000,1172.000000,1.441000e+03,1182.000000,1.354000e+03
75%,1.707605e+05,1321.250000,1410.000000,1154.000000,1081.000000,2703.250000,3.249250e+03,2756.250000,3.109000e+03
max,1.081691e+08,488263.000000,473286.000000,423681.000000,672754.000000,587625.000000,1.187700e+06,485504.000000,1.155810e+06


In [10]:
for i in df.columns:
    df[i].nunique()

9784

9779

9779

9776

3078

...

2814

4390

4762

4421

4688

In [11]:
df.columns

Index(['Ensembl_ID', 'SYMBOL', 'ENTREZID', 'GENENAME', 'F_IgE_S3',
       'F_unstim_S4', 'M_IgE_S1', 'M_unstim_S2', 'M_IgE_S6', 'M_unstim_S7',
       'F_IgE_S8', 'F_unstim_S9'],
      dtype='object')

In [12]:
df.to_csv('preprocessed_counts.csv', index=False)

# TMM normalization

### Select reference sample

In [13]:
def find_ref(df, cols):
    test = df[cols]
    
    quantiles = []
   
    for i in cols:
        scaled = test[i]/(test[i].sum())
        quantile = np.quantile(scaled, 0.75)
        quantiles.append(quantile)

    mean = np.mean(quantiles)
    ref = cols[list(abs(quantiles - mean)).index(min(abs(quantiles - mean)))]
    print('reference sample is:', ref)
    return ref

In [14]:
ref = find_ref(df, count_cols)

reference sample is: F_unstim_S4


## Find normalization factors

In [15]:
def find_scaling_factors(df, cols, ref):

    ref_sum = df[ref].sum()
    ref_ratio = df[ref] / ref_sum
    ref_var = (ref_sum - ref_ratio) / (ref_sum * ref_ratio)
    
    norm_factors = []
    
    for i in cols:
        i_sum =df[i].sum()
        i_ratio = df[i]/i_sum
        i_var = (i_sum - i_ratio) / (i_sum * i_ratio)
        
        weights = i_var + ref_var
        
        df1 = pd.DataFrame()
        
        df1['m'] = np.log2(i_ratio) - np.log2(ref_ratio)
        df1['a'] = np.log2(i_ratio * ref_ratio) * 0.5
        df1['weights'] = weights
        
        df1 = df1.replace(np.NINF, np.nan).replace(np.inf, np.nan).dropna()
        
        df1 = df1.loc[(df1['m'] >= np.quantile(df1['m'], .30)) & 
                               (df1['m'] <= np.quantile(df1['m'], .70)) &
                               (df1['a'] >= np.quantile(df1['a'], .05)) &
                               (df1['a'] <= np.quantile(df1['a'], .95))
                              ]

        df1['mxweights'] = df1['m'] * df1['weights']
        norm = (df1['mxweights'].sum()) / (df1['weights'].sum())
        norm_factors.append(2**norm)
    
    factors = pd.DataFrame()
    factors['samples'] = cols
    factors['raw factors'] = norm_factors
    
    adj = 1-np.mean(norm_factors)
    
    factors['centered factors'] =  factors['raw factors'] +adj
    display(factors)
    return factors[['samples', 'centered factors']]
    
    

In [16]:
factors = find_scaling_factors(df, count_cols, ref)

,samples,raw factors,centered factors
0,F_IgE_S3,0.929757,0.954005
1,F_unstim_S4,1.000000,1.024248
2,M_IgE_S1,0.910063,0.934311
3,M_unstim_S2,0.798981,0.823229
4,F_IgE_S8,1.045517,1.069765
5,F_unstim_S9,1.042354,1.066602
6,M_IgE_S6,1.030047,1.054295
7,M_unstim_S7,1.049298,1.073546


## Scale

In [17]:
for i in count_cols:
    weight = factors.loc[factors['samples']==i, 'centered factors'].values[0]
    df[i] = df[i] * weight

In [18]:
df

,Ensembl_ID,SYMBOL,ENTREZID,GENENAME,F_IgE_S3,F_unstim_S4,M_IgE_S1,M_unstim_S2,M_IgE_S6,M_unstim_S7,F_IgE_S8,F_unstim_S9
0,ENSMUSG00000000001,Gnai3,14679.0,guanine nucleotide binding protein (G protein)...,7125.462437,6642.248190,5196.637047,4897.390351,13288.332361,17125.204486,14769.169153,15890.233423
2,ENSMUSG00000000028,Cdc45,12544.0,cell division cycle 45,87.768449,310.347140,131.737832,265.903023,141.275511,115.942959,100.557866,149.324250
6,ENSMUSG00000000056,Narf,67608.0,nuclear prelamin A recognition factor,813.766161,1311.037422,588.615847,612.482505,1237.742161,2581.877933,2088.180370,2246.263363
7,ENSMUSG00000000058,Cav2,12390.0,caveolin 2,213.697093,208.946589,54.190030,155.590314,179.230125,230.812372,125.162450,252.784624
8,ENSMUSG00000000078,Klf6,23849.0,Kruppel-like factor 6,4537.247202,4556.879290,4360.428821,4238.807012,9768.041838,14904.037982,9842.903475,12732.025532
...,...,...,...,...,...,...,...,...,...,...,...,...
47643,ENSMUSG00000109865,Hspa14,50497.0,heat shock protein 14,1783.035118,1237.291567,1448.181845,1222.495322,2712.700664,1741.291479,2227.249759,1755.626541
47679,ENSMUSG00000109901,Chmp1b,67064.0,charged multivesicular body protein 1B,1661.876498,1420.631957,1185.640491,777.128339,3694.249174,4547.540509,3957.059010,4303.738211
47958,ENSMUSG00000110185,Igip,109169.0,IgA inducing protein,49.608254,91.158071,34.569502,23.873646,250.922176,293.078036,191.487852,245.318411
47978,ENSMUSG00000110206,Flt3l,14256.0,FMS-like tyrosine kinase 3 ligand,117.342600,489.590537,57.927274,161.352918,243.542112,1244.239719,289.906189,1094.333433


In [ ]:
df.to_csv('')

In [19]:
diff = pd.DataFrame(columns = ['ENTREZID', 'Fu', 'Fi', 'Mu', 'Mi', 'mean'])
diff['ENTREZID'] = df['ENTREZID']
diff['Fu'] = abs(df['F_unstim_S4'] - df['F_unstim_S9'])
diff['Fi'] = abs(df['F_IgE_S3'] - df['F_IgE_S8'])
diff['Mu'] = abs(df['M_unstim_S2'] - df['M_unstim_S7'])
diff['Mi'] = abs(df['M_IgE_S1'] - df['M_IgE_S6'])
diff['mean'] = diff[['Fu', 'Fi', 'Mu', 'Mi']].mean(axis = 1)
diff.sort_values('mean')

,ENTREZID,Fu,Fi,Mu,Mi,mean
2001,69534.0,1.605874,1.800348,11.615010,0.247789,3.817256
4445,71242.0,0.172985,3.013800,17.800650,17.570882,9.639579
17254,66128.0,17.346592,1.640465,6.457233,18.872966,11.079314
15898,108900.0,1.305211,12.050735,23.099854,13.190417,12.411554
8476,71876.0,13.642727,31.115053,3.920585,6.290289,13.742164
...,...,...,...,...,...,...
14251,14062.0,168825.860088,112553.080501,208953.547195,92858.979086,145797.866717
3555,17228.0,216236.155673,240117.511447,175782.559371,150949.634915,195771.465351
5337,17082.0,186601.633047,185460.768094,225515.434438,244827.522423,210601.339501
32021,72289.0,440662.705538,238531.149449,721575.734566,334410.537905,433795.031864
